In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time

import os


In [2]:
input_file="diy_rejection_21-May'25.csv"
output_file="diy_mco_rejection_20th May'25.csv"

In [3]:
#input_file="diy_all_rejection_01st April.csv"
#output_file="diy_mco_all_rejection_31st Mar'25.csv"

In [4]:
rejection_data=pd.read_csv(input_file,low_memory=False,memory_map=True)
len(rejection_data)

16434

In [5]:
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1007868460,e96c188d-5627-4b86-a664-0a85605af84e,INDIVIDUAL,Current Account not allowed for Individual mer...,BankStatement,2025-05-11 11:46:35.000,diy_mco,NaN,NaN
1,1355304800,b5422a9d-4d9d-4fac-b57d-58b331a7658f,PROPRIETORSHIP,Complete Shop photo not uploaded,shopFrontPhoto,2025-05-02 11:01:50.000,diy_mco,business_name_update,NaN
2,1866902951,ca723c0e-6b5a-4261-a613-fe5b6c6e9b69,PROPRIETORSHIP,Random/wrong photo,shopFrontPhoto,2025-05-15 21:21:33.000,diy_mco,diy_upgrade,NaN
3,1097297829,a2545e5b-77b4-421f-8e67-9ef64afa0f61,INDIVIDUAL,Beneficiary name/Account number/IFSC missing,BankStatement,2025-05-03 08:16:53.000,diy_mco,diy_upgrade,rekyc
4,500991630,a08aa18b-8536-4b5a-bb70-fd4ca611d1ac,PROPRIETORSHIP,Proprietor name is missing,udyam_udhyog_aadhaar,2025-05-13 12:47:02.000,diy_mco,diy_upgrade,NaN


In [6]:
rejection_data['updated_at']=pd.to_datetime(rejection_data['updated_at'],errors='coerce')
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1007868460,e96c188d-5627-4b86-a664-0a85605af84e,INDIVIDUAL,Current Account not allowed for Individual mer...,BankStatement,2025-05-11 11:46:35,diy_mco,NaN,NaN
1,1355304800,b5422a9d-4d9d-4fac-b57d-58b331a7658f,PROPRIETORSHIP,Complete Shop photo not uploaded,shopFrontPhoto,2025-05-02 11:01:50,diy_mco,business_name_update,NaN
2,1866902951,ca723c0e-6b5a-4261-a613-fe5b6c6e9b69,PROPRIETORSHIP,Random/wrong photo,shopFrontPhoto,2025-05-15 21:21:33,diy_mco,diy_upgrade,NaN
3,1097297829,a2545e5b-77b4-421f-8e67-9ef64afa0f61,INDIVIDUAL,Beneficiary name/Account number/IFSC missing,BankStatement,2025-05-03 08:16:53,diy_mco,diy_upgrade,rekyc
4,500991630,a08aa18b-8536-4b5a-bb70-fd4ca611d1ac,PROPRIETORSHIP,Proprietor name is missing,udyam_udhyog_aadhaar,2025-05-13 12:47:02,diy_mco,diy_upgrade,NaN


In [7]:
current_date = pd.Timestamp(datetime.now().date())
current_date


Timestamp('2025-05-21 00:00:00')

In [8]:
rejection_data=rejection_data[rejection_data['updated_at'].dt.date != current_date]
len(rejection_data)

16434

In [9]:
rejection_data=rejection_data[rejection_data['solution_type_level_2'].isin(['diy_upgrade','']) | rejection_data['solution_type_level_2'].isna()]
len(rejection_data)

9441

In [10]:
rejection_data.loc[~rejection_data['entity_type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'entity_type']='NON_PROPRIETORSHIP'
rejection_data['entity_type'].value_counts()

entity_type
INDIVIDUAL            6173
PROPRIETORSHIP        3249
NON_PROPRIETORSHIP      19
Name: count, dtype: int64

In [11]:
rejection_data.insert(rejection_data.columns.get_loc('entity_type')+1,'entuty_type_1',np.nan)
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [12]:
rejection_data.loc[:, 'entuty_type_1'] = rejection_data['entity_type']
rejection_data['entuty_type_1'].value_counts()

C:\Users\avnish1.yadav\AppData\Local\Temp\ipykernel_20600\2853859728.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['INDIVIDUAL' 'PROPRIETORSHIP' 'INDIVIDUAL' ... 'INDIVIDUAL' 'INDIVIDUAL'
 'INDIVIDUAL']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  rejection_data.loc[:, 'entuty_type_1'] = rejection_data['entity_type']


entuty_type_1
INDIVIDUAL            6173
PROPRIETORSHIP        3249
NON_PROPRIETORSHIP      19
Name: count, dtype: int64

In [13]:
rejection_data['entuty_type_1']=rejection_data['entuty_type_1'].replace(['PROPRIETORSHIP','NON_PROPRIETORSHIP'],'NON_EDC')
rejection_data['entuty_type_1'].value_counts()

entuty_type_1
INDIVIDUAL    6173
NON_EDC       3268
Name: count, dtype: int64

In [14]:
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [15]:
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_MCO'
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (~rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_ReKYC'
rejection_data.loc[~(rejection_data['entity_type']=='INDIVIDUAL'),'solution_type']='DIY_Upgrade'

In [16]:
rejection_data.to_csv(output_file, index=False)